In [ ]:
# Install necessary packages
!pip install transformers datasets torch scikit-learn nltk unidecode emoji emoticon_fix --use-deprecated=legacy-resolver
!export CUDA_LAUNCH_BLOCKING=1
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
import numpy as np
from nltk.stem.snowball import SnowballStemmer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
import torch
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.corpus import stopwords
from unidecode import unidecode
import emoji
from emoticon_fix import emoticon_fix
from imblearn.over_sampling import SMOTE
import torch.nn.functional as F

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/3

In [ ]:
# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
snow_stemmer = SnowballStemmer(language='english')

# Function to preprocess text
def preprocess_text(text):
    if pd.isna(text):  # Handle missing text
        return ""
    text = unidecode(text)  # Convert accented characters
    text = emoji.demojize(text)  # Convert emojis
    text = emoticon_fix.emoticon_fix(text)
    words = text.split()
    words = [snow_stemmer.stem(word) for word in words if word.lower() not in stop_words]
    return " ".join(words)

# Load dataset
file_path = "climaconvo.csv"
df = pd.read_csv(file_path)

# Rename columns for easier access
df.rename(columns={'Tweet': 'text', 'Stance (Support, Oppose, Neutral)': 'label'}, inplace=True)

#df['label'].replace([1, 2, 3], [0, 1, 2])
df['label'] = df['label'].replace(1, 0)
df['label'] = df['label'].replace(2, 1)
df['label'] = df['label'].replace(3, 2)

df.dropna(subset=['label'], inplace=True)
#print(df.head())

# Apply preprocessing
df['text'] = df['text'].apply(preprocess_text)

print(df['label'])
df = df[df['label'].isin([0, 1, 2])]

print(df['label'])

# Train-test split (keep original class proportions)
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


1        0.0
2        0.0
4        2.0
5        0.0
6        0.0
        ... 
15301    0.0
15302    0.0
15305    2.0
15307    2.0
15308    0.0
Name: label, Length: 10407, dtype: float64
1        0.0
2        0.0
4        2.0
5        0.0
6        0.0
        ... 
15301    0.0
15302    0.0
15305    2.0
15307    2.0
15308    0.0
Name: label, Length: 10407, dtype: float64


In [ ]:
# Tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Define dataset class
class ClimateDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(list(texts), padding=True, truncation=True, max_length=256, return_tensors="pt")
        self.labels = torch.tensor(list(labels), dtype=torch.long)  # FIXED: Convert Series to list

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# Load data (Ensure df_train and df_test are defined)
X_train = df_train[['text']]
y_train = df_train['label']

# Apply oversampling to balance the dataset
ros = RandomOverSampler(sampling_strategy="auto", random_state=42)
X_train_balanced, y_train_balanced = ros.fit_resample(X_train, y_train)

# Convert to lists for dataset creation
X_train_balanced = X_train_balanced['text'].tolist()
y_train_balanced = y_train_balanced.tolist()

# Compute class weights AFTER balancing
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train_balanced), y=y_train_balanced)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Create datasets
train_dataset = ClimateDataset(X_train_balanced, y_train_balanced, tokenizer)
val_dataset = ClimateDataset(df_test['text'], df_test['label'], tokenizer)

# Load pre-trained BERT model with increased dropout
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
model.config.hidden_dropout_prob = 0.3  # More dropout for generalization

# Define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=3e-4, momentum = 0.9)

# Define custom Trainer with Weighted Loss
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs["labels"]
        outputs = model(**inputs)
        logits = outputs.logits

        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# Define evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,  # Mixed precision for faster training
    report_to="none",
    save_on_each_node=False,
    disable_tqdm=False
)

# Trainer
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop if no improvement in 3 epochs
)

# Train model
trainer.train()

# Evaluate model
print("\nGenerating Classification Reports...\n" + "-" * 50)

# Train Set Classification Report
train_preds = trainer.predict(train_dataset).predictions
train_preds = np.argmax(train_preds, axis=1)
print("\nTrain Set Classification Report:")
print(classification_report(y_train_balanced, train_preds))

# Validation Set Classification Report
test_preds = trainer.predict(val_dataset).predictions
test_preds = np.argmax(test_preds, axis=1)
print("\nValidation Set Classification Report:")
print(classification_report(df_test['label'].tolist(), test_preds))

print("-" * 50)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-8df6e68d0d69>:85: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.735300,0.797457,0.646974
2,0.736400,0.739210,0.646494
3,0.701200,0.733643,0.644573
4,0.690500,0.714241,0.676273
5,0.706500,0.710611,0.663305



Generating Classification Reports...
--------------------------------------------------



Train Set Classification Report:
              precision    recall  f1-score   support

         0.0       0.58      0.71      0.64      4917
         1.0       0.90      0.80      0.85      4917
         2.0       0.59      0.52      0.55      4917

    accuracy                           0.68     14751
   macro avg       0.69      0.68      0.68     14751
weighted avg       0.69      0.68      0.68     14751




Validation Set Classification Report:
              precision    recall  f1-score   support

         0.0       0.76      0.73      0.74      1229
         1.0       0.70      0.80      0.75       199
         2.0       0.53      0.54      0.53       654

    accuracy                           0.68      2082
   macro avg       0.66      0.69      0.67      2082
weighted avg       0.68      0.68      0.68      2082

--------------------------------------------------


In [ ]:
# Extract features from roberta
def extract_roberta_features(model, dataloader, device):
    model.eval()
    features = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            hidden_states = outputs.hidden_states[-1]  # Last hidden layer
            features.append(hidden_states[:, 0, :].cpu().numpy())  # CLS token representation
    return np.vstack(features)

# Prepare DataLoader
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load roberta model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Extract features
train_features = extract_roberta_features(model, train_dataloader, device)
val_features = extract_roberta_features(model, val_dataloader, device)

# Normalize features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
val_features = scaler.transform(val_features)

# --------------------------------
# 🔹 Define Nested BiLSTM Model
# --------------------------------

class NestedBiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
        super(NestedBiLSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim

        # First BiLSTM layer
        self.bilstm1 = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True, dropout=dropout, bidirectional=True)

        # Subsequent BiLSTM layers
        self.bilstm_layers = nn.ModuleList([
            nn.LSTM(hidden_dim * 2, hidden_dim, num_layers=1, batch_first=True, dropout=dropout, bidirectional=True)
            for _ in range(num_layers - 1)
        ])

        # Fully connected layer (adjusted for bidirectional hidden size)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        # Pass through first BiLSTM layer
        out, _ = self.bilstm1(x)

        # Pass through subsequent BiLSTM layers (nested)
        for bilstm in self.bilstm_layers:
            out, _ = bilstm(out)

        # Get the last time-step output
        out = out[:, -1, :]  # Take the last time step

        # Fully connected layer
        output = self.fc(out)
        return output

# --------------------------------
# 🔹 Define Dataset Class for LSTM
# --------------------------------

class LSTMFeatureDataset(Dataset):
    def __init__(self, features, labels):
        # Add an extra dimension for sequence length (1 in this case)
        self.features = torch.tensor(features, dtype=torch.float32).unsqueeze(1)  # Shape: (batch_size, 1, input_dim)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# --------------------------------
# 🔹 Main Pipeline
# --------------------------------
# Prepare Dataset for Nested BiLSTM
train_lstm_dataset = LSTMFeatureDataset(train_features, y_train_balanced)
val_lstm_dataset = LSTMFeatureDataset(val_features, df_test['label'].tolist())

train_loader = DataLoader(train_lstm_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_lstm_dataset, batch_size=batch_size, shuffle=False)

# Define parameters for Nested BiLSTM
bilstm_input_dim = train_features.shape[1]  # Feature size from roberta
bilstm_hidden_dim = 256
bilstm_output_dim = 3  # Multi-class classification (e.g., Support, Denial, Neutral)
bilstm_num_layers = 1  # Number of BiLSTM layers (nested)
bilstm_dropout = 0.5

# Initialize Nested BiLSTM model
nested_bilstm_model = NestedBiLSTM(bilstm_input_dim, bilstm_hidden_dim, bilstm_output_dim, bilstm_num_layers, bilstm_dropout).to(device)

# Define optimizer & loss function for BiLSTM
bilstm_criterion = nn.CrossEntropyLoss()
bilstm_optimizer = torch.optim.Adam(nested_bilstm_model.parameters(), lr=2e-5)

# Train Nested BiLSTM model
for epoch in range(12):
    nested_bilstm_model.train()
    total_loss = 0

    for batch_features, batch_labels in train_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        bilstm_optimizer.zero_grad()
        outputs = nested_bilstm_model(batch_features)
        loss = bilstm_criterion(outputs, batch_labels)
        loss.backward()
        bilstm_optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/12], Loss: {total_loss / len(train_loader):.4f}")

# --------------------------------
# 🔹 Evaluate Nested BiLSTM Model
# --------------------------------

nested_bilstm_model.eval()
all_features, all_labels = [], []

with torch.no_grad():
    for batch_features, batch_labels in val_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        # Get Nested BiLSTM output (features before classification layer)
        outputs = nested_bilstm_model(batch_features)

        # Collect features and labels
        all_features.extend(outputs.cpu().numpy())
        all_labels.extend(batch_labels.cpu().numpy())

# Print classification report after Nested BiLSTM
print("\nValidation Set Classification Report (After Nested BiLSTM):")
print(classification_report(all_labels, np.argmax(all_features, axis=1)))

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


Epoch [1/12], Loss: 0.7278
Epoch [2/12], Loss: 0.6451
Epoch [3/12], Loss: 0.6252
Epoch [4/12], Loss: 0.6084
Epoch [5/12], Loss: 0.5920
Epoch [6/12], Loss: 0.5757
Epoch [7/12], Loss: 0.5595
Epoch [8/12], Loss: 0.5436
Epoch [9/12], Loss: 0.5278
Epoch [10/12], Loss: 0.5126
Epoch [11/12], Loss: 0.4984
Epoch [12/12], Loss: 0.4842

Validation Set Classification Report (After Nested BiLSTM):
              precision    recall  f1-score   support

           0       0.77      0.68      0.72      1229
           1       0.77      0.81      0.79       199
           2       0.50      0.62      0.55       654

    accuracy                           0.67      2082
   macro avg       0.68      0.70      0.69      2082
weighted avg       0.69      0.67      0.68      2082



In [ ]:
# --------------------------------
# 🔹 MLP Training
# --------------------------------

# Train MLP on BiLSTM features
mlp_classifier = MLPClassifier(hidden_layer_sizes=(1024,512, 256), max_iter=1000, random_state=42, solver='adam', learning_rate_init=1e-3)
mlp_classifier.fit(all_features, all_labels)  # Train on features, not labels and preds

# Predictions from MLP
y_pred_mlp = mlp_classifier.predict(all_features)  # Predict on the features extracted from BiLSTM

# Classification Report
print("\nValidation Set Classification Report for MLP:")
print(classification_report(all_labels, y_pred_mlp))  # Compare true labels and predicted labels

# Save Predictions to CSV
df_predictions = pd.DataFrame({
    "Actual": all_labels,  # Actual true labels
    "Predicted": y_pred_mlp  # Predicted labels from MLP
})
prediction_file_path = "stance_predictions_roberta.csv"
df_predictions.to_csv(prediction_file_path, index=False)

print(f"Predictions saved as {prediction_file_path}!")


Validation Set Classification Report for MLP:
              precision    recall  f1-score   support

           0       0.73      0.89      0.80      1229
           1       0.99      0.75      0.85       199
           2       0.64      0.42      0.51       654

    accuracy                           0.73      2082
   macro avg       0.79      0.69      0.72      2082
weighted avg       0.73      0.73      0.71      2082

Predictions saved as stance_predictions_roberta.csv!
